## feedback 

In [ ]:
# fixed variables

# feedback parameters
FB_TARGET = 1e-5  # give in terms of power output
FB_STEPSIZE = 1e-4
FB_SLOPE = 1  # 1 or -1 depending on the slope to lock onto

# pulse parameters
MIXED_AMP = {  # voltage amplitude for mixed state preperation
    "P1": 0.1,
    "P2": 0.1,
}
MES_AMP_START = {  # voltages for stepping over when sweeping
    "P1": 0.1,
    "P2": 0.2,
    "J": ...,
}
MES_AMP_END = {
    "P1": 0.2,
    "P2": 0.1,
    "J": ...,
}

# number of measurement steps in detuning and J
NUM_DETUNING = 100
NUM_J = 1  


In [ ]:
# adjust variables
curr_st_amp = 0.3

# current command tables
cts = {c: CommandTable(chan[c].awg.commandtable.load_validation_schema()) for c in drive_chans}

j_steps = np.linspace(MES_AMP_START["J"], MES_AMP_END["J"], NUM_J)
p1_steps = np.linspace(MES_AMP_START["P1"], MES_AMP_END["P1"], NUM_DETUNING)
p2_steps = np.linspace(MES_AMP_START["P2"], MES_AMP_END["P2"], NUM_DETUNING)

### Setup command tables

In [ ]:
def cmdtable(ct, amplitude, length, wave_index, ct_index):
    """
    Load a default command table with a sin/cos wave (used throughout the documentation)
    """
    ct.table[ct_index].waveform.index = wave_index
    ct.table[ct_index].amplitude00.value = amplitude  # all in dBm
    ct.table[ct_index].amplitude01.value = -amplitude
    ct.table[ct_index].amplitude10.value = amplitude
    ct.table[ct_index].amplitude11.value = amplitude
    ct.table[ct_index].waveform.length = length  # in samples
    ct.table[ct_index].waveform.samplingRateDivider = samplingDivider  # inherit global

In [ ]:
# each channel only needs two pulses.
# initial mixed state creation and then measurement pulse

# INDEX
# 0 => Create mixed state
# 1 => Measurement

chan["P1"].awg.commandtable.upload_to_device(ct)

In [ ]:
# load mixed state command channels
for c in ["P1", "P2"]:
    cmdtable(cts[c],
             amplitude= voltToDbm(MIXED_AMP[c], chan[c].output.range()),
             length=timeToSamples(..., samplingDivider),
             wave_index=0,
             ct_index=0,
            )

In [ ]:
def uploadcts():
    """Upload the command tables to the device"""
    for c in drive_chans:
        chan[c].awg.commandtable.upload_to_device(cts[c])

### Run functions

In [ ]:
def waitForInternalTrigger():
    """
    Waits for the internal trigger to finish running and shows the current progress.
    """
    pbar = tqdm(total=100)
    while device.system.internaltrigger.progress() != 1.0:
        p = int(device.system.internaltrigger.progress()*100)
        pbar.update(p-pbar.n)
        time.sleep(0.001)
    pbar.update(100-pbar.n)
    pbar.close()

In [ ]:
def calculatefeedback(data):
    """Calculate the appropriate feedback adjustment to ST."""
    # use last datapoint
    error = FB_TARGET-np.abs(data[-1])
    curr_st_amp += error*FB_STEPSIZE*FB_SLOPE

    # update command table
    cmdtable(cts["ST"],
             amplitude= voltToDbm(curr_st_amp, chan["ST"].output.range()),
             length=timeToSamples(..., samplingDivider),
             wave_index=0,
             ct_index=0,
            )
    

In [ ]:
def movemeasurement(i):
    """Move P1/P2/J to measure the next appropriate datapoint."""
    

In [ ]:
def runexperiment():
    device.system.internaltrigger.enable(0)

    result_node = chan["measure"].spectroscopy.result.data.wave
    result_node.subscribe()
    
    chan["measure"].spectroscopy.result.enable(1)  # start logger
    
    
    # start sequencers
    chan["measure"].generator.enable_sequencer(single=True)
    chan["J"].awg.enable_sequencer(single=True)  # dont want to repeat
    chan["P1"].awg.enable_sequencer(single=True)
    chan["P2"].awg.enable_sequencer(single=True)
    
    
    # start triggering sequence (which starts each sequencer)
    device.system.internaltrigger.enable(1)
    time.sleep(0.2)  # delay for networking issues
    
        
    # wait for the measurement to complete
    waitForInternalTrigger()
    
    
    # check sequencers have finished their sequences
    # Status of the Sequencer on the instrument.
    # - Bit 0: Sequencer is running;
    # - Bit 1: reserved;
    # - Bit 2: Sequencer is waiting for a trigger to arrive;
    # - Bit 3: Sequencer has detected an error;
    # - Bit 4: sequencer is waiting for synchronization with other channels
    m_state = chan["measure"].generator.sequencer.status()
    st_state = chan["P1"].awg.sequencer.status()
    if m_state != 4 and st_state != 4:
        print(TimeoutError(f"Sequencers in unknown state. Perhaps they are not synchronised? {bin(m_state)}, {bin(st_state)}"))
        time.sleep(0.5)
    
    
    # wait for completion
    while chan["measure"].spectroscopy.result.enable() != 0:
        print(chan["measure"].spectroscopy.result.enable())
        chan["measure"].spectroscopy.result.enable.wait_for_state_change(0, timeout=10)
    
    
    # get results
    results = get_results(result_node, timeout=5)
    result_node.unsubscribe()
    
    
    # verify results
    acq = chan["measure"].spectroscopy.result.acquired()
    if len(results) > acq:
        print(chan["measure"].generator.ready())
        print([chan[c].awg.ready() for c in drive_chans])
        raise TimeoutError(f"Not all datapoints measured in the time provided. {acq} of {len(results)}.")
    
    return np.mean(results.reshape((seq_averages, ...)), axis=0)




On each datapoint need to 
 - Measure the two reference points
 - Average from SEQC
 - Apply feedback to negate offset
 - Adjust command tables for next datapoint

### Run

In [ ]:
monty.newrun("feedback", params)

for j in j_steps:
    for (p1, p2) in zip(p1_steps, p2_steps):
        movemeasurement(p1, p2, j)  # CT = 1
        calculatefeedback(data)  # CT = 0
        uploadcts()
        data = runexperiment()
        

plt.plot(np.linspace(0, pulse_time, read_lens), np.abs(data))  
plt.ylabel("RF signal (arb units)")
plt.title(monty.identifier + "." + monty.runname)

monty.save({"data": data})
monty.savefig(plt, "ramp decay")

In [ ]:
# calculate feedback

# take abs of last measurement
error = 